# Stations Discovery + Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET stations, where:

- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

for each discovered station, filter all acquisitions where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h

In [ ]:
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

json.dump(item_dict, sys.stdout, indent=2)

## Execute the `query_stations_from_parquet` operation

In [ ]:

source_item: Item = Item.from_dict(item_dict)
cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                source_item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

In [ ]:
from pygeofilter_aeronet import (
    DEFAULT_STATIONS_PARQUET_URL,
    query_stations_from_parquet
)
from typing import List

_, resulting_items = query_stations_from_parquet(
    cql2_filter=cql2_filter,
    file_path=DEFAULT_STATIONS_PARQUET_URL
)

## Prepare the Map screen

In [ ]:
print(f"Found {len(resulting_items)} STAC Item(s) representing AERONET Station(s)")

In [ ]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import Fullscreen

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=source_item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

for item in resulting_items:
    GeoJson(
        item,
        name=item.id,
        style_function=lambda f: {
            "fillColor": "yellow",
            "color": "red",
            "weight": 2,
            "fillOpacity": 0.3,
        }
).add_to(map)

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map

## `search` data for each discovered station

In [ ]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame
from IPython.display import (
    display,
    Markdown
)
from pygeofilter_aeronet import aeronet_search

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(source_item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

for item in resulting_items:
    site_name: str = item.properties['aeronet:site_name']

    cql2_filter = {
        "op": "and",
        "args": [
            {"op": "eq", "args": [{"property": "site"}, site_name]},
            {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
            {"op": "eq", "args": [{"property": "format"}, "csv"]},
            {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
            {
                "op": "t_after",
                "args": [
                    {"property": "time"},
                    {"timestamp": _serialize_date(start_datetime)},
                ],
            },
            {
                "op": "t_before",
                "args": [
                    {"property": "time"},
                    {"timestamp": _serialize_date(end_datetime)},
                ],
            }
        ],
    }

    try:
        resulting_item: Item = aeronet_search(
            cql2_filter=cql2_filter,
            output_dir=out_dir
        )

        display(Markdown(f"### {site_name} results as Data Frame"))

        geoparquet_file: str = resulting_item.get_assets()['geoparquet'].href
        geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

        display(geoparquet_data)
    except Exception as e:
        print(f"An error occurred while retrieving data from site {site_name}: {e}")

An error occurred while retrieving data from site La_Crau: No columns to parse from file


2025-11-17 15:48:35.898 | SUCCESS  | pygeofilter_aeronet:aeronet_search:235 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-17 15:48:35.907 | SUCCESS  | pygeofilter_aeronet:aeronet_search:248 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/40e66e5c-c1e0-4353-badd-13ca6d38b10f.csv
2025-11-17 15:48:35.918 | SUCCESS  | pygeofilter_aeronet:aeronet_search:282 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/40e66e5c-c1e0-4353-badd-13ca6d38b10f.parquet


### Lille results as Data Frame

An error occurred while retrieving data from site JUNGFRAU: No columns to parse from file
An error occurred while retrieving data from site Aire_Adour: No columns to parse from file
An error occurred while retrieving data from site Venise: No columns to parse from file


KeyboardInterrupt: 